# 라이브러리 설치

In [5]:
!pip install mysql-connector-python
!pip show sqlalchemy

Name: SQLAlchemy
Version: 2.0.31
Summary: Database Abstraction Library
Home-page: https://www.sqlalchemy.org
Author: Mike Bayer
Author-email: mike_mp@zzzcomputing.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: greenlet, typing-extensions
Required-by: bigframes, ipython-sql


# 라이브러리 선언

In [6]:
import pandas as pd
from sqlalchemy import create_engine, inspect

# 구글드라이브 연동

In [7]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [8]:
%cd /content/gdrive/MyDrive/Colab Notebooks/analysis_edu/05. 데이터분석 및 데이터 전처리

/content/gdrive/MyDrive/Colab Notebooks/analysis_edu/05. 데이터분석 및 데이터 전처리


# 파일 불러오기

In [9]:
googleUrl = "../dataset/customerdata.csv"
csDataGitUrl = "https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/customerdata.csv"

In [10]:
customerData = pd.read_csv(csDataGitUrl)
customerData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CUSTID       150 non-null    object 
 1   AVGPRICE     150 non-null    float64
 2   EMI          150 non-null    int64  
 3   DEVICECOUNT  150 non-null    int64  
 4   PRODUCTAGE   150 non-null    float64
 5   CUSTTYPE     150 non-null    object 
dtypes: float64(2), int64(2), object(2)
memory usage: 7.2+ KB


In [11]:
# 테이블정의서 저장
typeDf  = pd.DataFrame( customerData.dtypes )
typeDf.to_csv("./테이블정의서.csv", index=True)

### 1. 원하는 행 조회하기 (기본조건)

In [12]:
emiCond = 2
avgPriceCond = 3500

In [13]:
customerData.loc[  (customerData.EMI > emiCond) &
                   (customerData.AVGPRICE >= avgPriceCond) ].shape

(11, 6)

In [14]:
# ★ 조건검증 꼭 해보기
customerData.loc[  (customerData.EMI <= emiCond) |
                   (customerData.AVGPRICE < avgPriceCond) ].shape

(139, 6)

### 1. 원하는 행 조회하기 (in not in 조건)

In [15]:
emiMultiCond = [2,3]

In [16]:
customerData.loc[ ~ customerData.EMI.isin( emiMultiCond )  ].head(2)

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
11,C18622,3246.000000,1,5,1.891624,Big-Screen-lover
46,I14835,3374.091339,1,4,2.749678,Sleeping-dog


### 1. 원하는 행 조회하기 (컬럼 값 세부 조건)

In [17]:
 customerData.loc [customerData.CUSTID.astype(str).str[0] == "A"]

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
0,A13566,4273.900000,3,6,1.679181,Big-Screen-lover
1,A14219,3642.441950,2,4,2.682023,Sleeping-dog
2,A15312,3653.884565,2,5,3.208202,Sleeping-dog
3,A16605,3713.211107,2,6,0.900000,Early-bird


### 컬럼명 조작방법

In [18]:
# 데이터프레임 컬럼 정보를 시연!
customerData.columns

Index(['CUSTID', 'AVGPRICE', 'EMI', 'DEVICECOUNT', 'PRODUCTAGE', 'CUSTTYPE'], dtype='object')

In [19]:
# 데이터프레임 컬럼정보 수정 * 문자열함수를 사용하려면 .str을 df에서는 써야한다!
customerData.columns = customerData.columns.str.upper()

In [20]:
## 전체 데이터 프레임의 크기 확인용
customerData.shape

(150, 6)

### 2. 데이터 타입변환 (astype)

In [21]:
customerData.EMI = customerData.EMI.astype(str)

In [22]:
columnNameList = list( customerData.columns )

In [23]:
for i in range(0, len(columnNameList)):
    customerData[ columnNameList[i] ] = customerData[ columnNameList[i] ].astype(str)

### 3. 인덱스 초기화

In [24]:
targetColumns = ["AVGPRICE","EMI"]
# 컬럼 이름으로 조회
customerDataIndex = customerData.loc[ customerData.AVGPRICE.astype(float) < 3000 ,
                  targetColumns ].reset_index(drop=True)

[데이터타입 조회]
kopo_customerdata.csv 파일을 불러온 후
kopo_customerData 변수에 담으세요 이후
STATENAME, GENDER 컬럼 첫 5개행 조회하세요

In [25]:
csDataLocalUrl = "../dataset/kopo_customerdata.csv"
csDataGitUrl = "https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/kopo_customerdata.csv"

In [26]:
# 데이터 기본조회
# 데이터프레임명.loc[ 조건 ]
# 단 조건은 데이터프레임명.컬럼명에 대한 조건 기입해야함
# 또한 각 컬럼값의 문자열 함수를 사용하고 싶은 경우
# 데이터프레임명.str.upper() 형태로 사용 가능 (예: .str.replace("/","-") 등)
kopo_customerData.loc[ kopo_customerData.GENDER.str.upper()=="FEMALE",
                       targetColumns]

NameError: name 'kopo_customerData' is not defined

In [27]:
kopo_customerData = pd.read_csv(csDataGitUrl)
targetColumns = ["STATENAME","GENDER"]
kopo_customerData.loc[ 0:4, targetColumns]

,STATENAME,GENDER
0,State2,Male
1,State2,Male
2,State2,Female
3,State2,Male
4,State2,Male


### 4. 데이터 조작하기

In [28]:
import numpy as np

In [29]:
customerData["PRODUCTAGE_NEW"]=\
    np.where(  customerData.PRODUCTAGE.astype(float) < 1, 1,  customerData.PRODUCTAGE)
# 내가 작성한 로직 검증로직
customerData.loc[ (customerData.PRODUCTAGE.astype(float) < 1) &
                  (customerData.PRODUCTAGE_NEW != 1) ]
# 내가 작성한 로직 검증로직
customerData.loc[ (customerData.PRODUCTAGE.astype(float) >= 1) &
                  (customerData.PRODUCTAGE_NEW != customerData.PRODUCTAGE) ]

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,PRODUCTAGE_NEW


In [30]:
### NP.WHERE  활용 데이터 조작 ( 간단 로직 구현 시)
# customerData_new < 1 = 1
customerData["PRODUCTAGE_NEW"]=\
    np.where(  customerData.PRODUCTAGE.astype(float) < 1, 1,
             np.where(  customerData.PRODUCTAGE.astype(float) < 2, 2,
                        customerData.PRODUCTAGE))

In [31]:
### 함수 활용 데이터 조작 ( 복잡 로직 구현 시)
def productAgeCal( inValue = 0.5 ):
    """
    정의: 값을 입력받아 1보다 작으면 1, 2보다 작으면 2, 이외에는 3값을 리턴
    """
    # inValue = 5
    inValue = float(inValue)
    outValue = 1
    # 1보다 작으면 1, 2보다 작으면 2, 이외에는 3
    if inValue < 1:
        outValue = 1
    elif inValue < 2:
        outValue = 2
    else:
        outValue = 3
    return outValue
customerData["PRODUCTAGE_NEW"] = customerData.PRODUCTAGE.apply( productAgeCal )

### loc 활용 데이터 조작하기

In [32]:
### LOC 활용 데이터 조작 [특정 데이터만 선택 후 조작 시 ]
customerData.loc[ customerData.PRODUCTAGE.astype(float) < 1, "PRODUCTAGE_NEW" ] = 1
customerData.loc[ (customerData.PRODUCTAGE.astype(float) >= 1) &
                  (customerData.PRODUCTAGE.astype(float) < 2), "PRODUCTAGE_NEW" ] = 2
customerData.loc[ customerData.PRODUCTAGE.astype(float) >= 2, "PRODUCTAGE_NEW" ] = 3

In [33]:
targetColums = ["PRODUCTAGE" ]
# 조건에 맞는 데이터 조회
customerData.loc[ customerData.PRODUCTAGE.astype(float) < 1, targetColums ]
# 조건에 맞는 특정 값 가져오기
customerData.loc[ customerData.PRODUCTAGE.astype(float) < 1, targetColums ].values[0][0]

'0.9'

In [34]:
csA13566avgPrice = customerData.loc[ customerData.CUSTID =="A13566", targetColums ].values[0][0]

### 5. 데이터 컬럼간 연산하기

In [35]:
### 컬럼간 데이터 연산 (조작)
customerData["NEW_COLUMN"] = customerData.CUSTID.str[1:] + "_" + customerData.EMI
### 컬럼 생성 및 중복값 저장
customerData["NEW_COLUMN2"] = 111
customerData = customerData.drop(columns= ["NEW_COLUMN2"] )

In [36]:
### 검증

In [37]:
# AVGPRICE < 3000 - low  3000 미만은 전부 low로 변경했다
                   # 3000 미만에서 low 여야 한다
                   # 3000 이상에서는 low 가 존재하면 안된다
# AVGPRICE 3000>=  400 <= mid
# 4000>=  high
customerData["CUST_SEG"] = np.where(customerData.AVGPRICE.astype(float) < 3000, "low",
         np.where(customerData.AVGPRICE.astype(float) < 4000, "mid", "high"))

In [38]:
customerData.loc[ (customerData.AVGPRICE.astype(float) < 3000) &
                  (customerData.CUST_SEG != "low") ]

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,PRODUCTAGE_NEW,NEW_COLUMN,CUST_SEG


In [39]:
customerData.loc[ (customerData.AVGPRICE.astype(float) >= 3000) &
                  (customerData.CUST_SEG == "low") ]

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,PRODUCTAGE_NEW,NEW_COLUMN,CUST_SEG


### CUST ID 머신러닝용 제대로 라벨링 하기 (우선순위에 따른 라벨링)

In [40]:
groupData = customerData.groupby(by=["CUSTTYPE"])["AVGPRICE"].agg(["mean"]).reset_index()
groupData
groupData = groupData.rename(columns={"mean":"MEAN"})
sortedData = groupData.sort_values(by=["MEAN"], ascending=True)
sortedData
labelList = list( sortedData.CUSTTYPE )
labelDict = {}
for i in range(0, len(labelList)):
    labelDict[labelList[i]] = i
labelDict

TypeError: agg function failed [how->mean,dtype->object]

### 6. 데이터 정렬하기

In [41]:
sortKey = ["EMI"]

In [42]:
sortedData = customerData.sort_values(by=sortKey, ascending = False)

In [43]:
groupKey = ["EMI","CUSTTYPE"]

### 이동평균 계산하기

In [45]:
selloutUrl = "https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/kopo_channel_seasonality_new.csv"
selloutData = pd.read_csv(selloutUrl)

In [58]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]

In [62]:
sortedData = selloutData.sort_values( by= sortKey ).reset_index(drop=True)
sortedData.head(2)

,REGIONID,PRODUCT,YEARWEEK,QTY,MA5
0,A00,PRODUCT34,201401,661.0,4338.0
1,A00,PRODUCT34,201402,679.0,4120.6


In [76]:
sortedData.loc[0:2, ["QTY"]].mean()

,0
QTY,639.333333


### 이동평균

In [119]:
# 심플 GROUP MA5 계산
def groupRolling(eachGroup):
    # 디버깅 코드
    # 1개 그룹 데이터만 선택 eachGroup = sortedData.loc[ 지역==A00 & 상품=PRODUCT ]
    # eachGroup = groupData.get_group( list(groupData.groups)[0] )
    eachGroup["GROUP_MA5"] = eachGroup.QTY.rolling(window=5, center=True, min_periods=1).mean()
    return eachGroup
groupKey = ["REGIONID","PRODUCT"]
groupMaData = sortedData.groupby(by=groupKey).apply(groupRolling).reset_index(drop=True)

# 심플 MA5 계산
groupMaData["SIMPLE_MA5"] = groupMaData.QTY.rolling(window=5, center=True, min_periods=1).mean()
groupMaData.to_csv("/content/ma_result.csv")

In [120]:
groupMaData

,REGIONID,PRODUCT,YEARWEEK,QTY,SIMPLE_MA5,GROUP_MA5
0,A00,PRODUCT34,201401,661.0,639.333333,639.333333
1,A00,PRODUCT34,201402,679.0,612.500000,612.500000
2,A00,PRODUCT34,201403,578.0,593.200000,593.200000
3,A00,PRODUCT34,201404,532.0,545.600000,545.600000
4,A00,PRODUCT34,201405,516.0,491.200000,491.200000
...,...,...,...,...,...,...
124653,A77,PRODUCT12,201648,4152.0,4582.600000,4582.600000
124654,A77,PRODUCT12,201649,5086.0,4718.200000,4718.200000
124655,A77,PRODUCT12,201650,5846.0,5513.600000,5513.600000
124656,A77,PRODUCT12,201651,4933.0,5854.000000,5854.000000


### 핵심 데이터 조회를 통해 값을 받아 처리

In [ ]:
paramDf = pd.read_csv("./paramData.txt")
ipName = "HOST"
portName = "PORT"
userIdName = "USERID"
userPassName = "USERPASS"
dbName = "DBNAME"
targetColum = ["PARAM_VALUE"]
dbIp = paramDf.loc[ paramDf.PARAM_NAME == ipName,targetColum].values[0][0]
dbPort = paramDf.loc[ paramDf.PARAM_NAME == portName,targetColum].values[0][0]
dbUserName = paramDf.loc[ paramDf.PARAM_NAME == userIdName,targetColum].values[0][0]
dbUserPass = paramDf.loc[ paramDf.PARAM_NAME == userPassName,targetColum].values[0][0]
dbName = paramDf.loc[ paramDf.PARAM_NAME == dbName,targetColum].values[0][0]

# DB 데이터 저장 (MySQL)

In [ ]:
# MySQL 데이터베이스 연결 정보 설정 (클라우드)
host = dbIp
port = dbPort
user = dbUserName
password = dbUserPass
database = dbName

# SQLAlchemy 엔진 생성
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}')

# 데이터프레임을 MySQL 데이터베이스의 테이블에 저장
table_name = 'paramdf_0731_final'
paramDf.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

5